In [771]:
import nltk
import re
from nltk.tokenize import sent_tokenize
import pandas as pd

In [772]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [773]:
conectivos = [
    "para concluir",
    "em resumo",
    "em suma",
    "portanto",
    "assim",
    "logo",
    "por conseguinte",
    "concluindo",
    "em conclusão",
    "em consequência",
    "deste modo",
    "por este motivo",
    "desse modo",
    "dessa forma",
    "consequentemente",
    "dessarte",
    "destarte",
    "em vista disso",
    "em vista do exposto",
    "em razão disso",
    "em virtude disso",
    "em decorrência disso",
    "em síntese",
    "em poucas palavras",
    "para finalizar",
    "por fim",
    "em última análise",
    "finalizando",
    "assim sendo",
    "dessa maneira",
    "dessa sorte",
    "dessa ordem de ideias",
    "em decorrência",
    "por essa razão",
    "por esse motivo",
    "de modo que",
    "de forma que",
    "enfim",
    "para sintetizar",
    "para resumir",
    "para encerrar",
    "diante disso",
    "tendo em vista",
    "considerando o exposto",
    "posto isso",
    "isso posto",
    "com base nisso",
    "conclui-se que",
    "conclui-se",
    "é possível concluir que",
    "é necessário concluir que",
    "a partir disso",
    "como resultado",
    "tudo considerado",
    "à guisa de conclusão",
    "urge que",
    "espera-se que",
    "sendo assim",
    "dessa maneira",
    "à luz do exposto",
    "à vista disso",
    "em face do exposto",
    "em virtude do exposto",
    "torna-se evidente que",
    "torna-se claro que",
    "porém",
    "é necessário",
]

In [774]:
# def extrair_trecho_final(texto, conectivos, limite=150):
#     trecho = texto[int(len(texto) * 0.60):].strip()
#     trecho_lower = trecho.lower()

#     # 1. Procura por todos os pontos dentro do limite e tenta achar conectivo depois
#     for i, char in enumerate(trecho[:limite]):
#         if char == '.':
#             trecho_pos_ponto = trecho[i + 1:].lstrip()
#             trecho_pos_ponto_lower = trecho_pos_ponto.lower()
#             for conjuncao in conectivos:
#                 pos_conj = trecho_pos_ponto_lower.find(conjuncao)
#                 if 0 <= pos_conj <= 30:  # tolera até 30 caracteres depois do ponto
#                     print(f"Conectivo encontrado após ponto: {conjuncao}")
#                     return trecho_pos_ponto[pos_conj:].strip()

#     # 2. Se não encontrou conectivo após ponto, procura diretamente no trecho
#     for conjuncao in conectivos:
#         pos_conj = trecho_lower.find(conjuncao)
#         if pos_conj != -1:
#             print(f"Conectivo encontrado direto: {conjuncao}")
#             return trecho[pos_conj:].strip()

#     # 3. Se não houver conectivo, tenta após vírgula
#     pos_virgula = trecho.find(',')
#     if pos_virgula != -1 and pos_virgula + 1 < len(trecho):
#         trecho_pos_virgula = trecho[pos_virgula + 1:].lstrip()
#         print("Nenhum conectivo ou ponto. Usando vírgula como fallback.")
#         return trecho_pos_virgula

#     # 4. Se nada funcionar, retorna o trecho inteiro
#     print("Nenhum conectivo encontrado, retornando trecho final.")
#     return trecho.strip()

In [775]:
def extrair_trecho_final(texto, conectivos, limite=80):
    trecho = texto[int(len(texto) * 0.70):].strip()
    trecho_lower = trecho.lower()
    max = 1000

    # 1. Procura por ponto e conectivo após o ponto
    for i, char in enumerate(trecho):
        if char == '.' or char == '!' or char == '?':
            trecho_pos_ponto = trecho[i + 1:].lstrip()
            palavras = trecho_pos_ponto.split()
            if palavras:
                primeira_palavra = palavras[0].strip('.,;!?')  # remove pontuação colada
                for conjuncao in conectivos:
                    if primeira_palavra.lower() == conjuncao:
                        print(f"[1] Conectivo após ponto: {conjuncao}")
                        return trecho_pos_ponto.strip()

    # 2. Busca conectivo no trecho inteiro e salva informações
    prioridade_Maiuscula = []
    prioridade_minuscula = []

    for conjuncao in conectivos:
        pos = trecho_lower.find(conjuncao)
        if pos != -1:
            original_conectivo = trecho[pos:pos + len(conjuncao)]
            if original_conectivo[0].isupper():
                prioridade_Maiuscula.append((pos, conjuncao))
            else:
                prioridade_minuscula.append((pos, conjuncao))

    # Ordena cada lista pela posição no texto
    prioridade_Maiuscula.sort()
    prioridade_minuscula.sort()

    if prioridade_Maiuscula:
        index = -2 if len(prioridade_Maiuscula) >= 2 else -1
        pos_conj, conjuncao = prioridade_Maiuscula[index]
        if limite < len(trecho[pos_conj:]) < max:
            print(f"[2] Conectivo Maiúsculo encontrado: {conjuncao}")
            return trecho[pos_conj:].strip()
    elif prioridade_minuscula:
        index = -2 if len(prioridade_minuscula) >= 2 else -1
        pos_conj, conjuncao = prioridade_minuscula[index]
        if limite < len(trecho[pos_conj:]) < max:
            print(f"[2] Conectivo minúsculo encontrado: {conjuncao}")
            return trecho[pos_conj:].strip()
        
    # 3. Fallback: procura após o primeiro ponto
    pos_ponto = trecho.find('.')
    if pos_ponto != -1 and pos_ponto + 1 < len(trecho):
        trecho_pos_ponto = trecho[pos_ponto + 1:].lstrip()
        if limite < len(trecho_pos_ponto) < max:
            print("[3] Fallback: ponto usado.")
            return trecho_pos_ponto
        
    # 4. Fallback: procura após a primeira vírgula
    pos_virgula = trecho.find(',')
    if pos_virgula != -1 and pos_virgula + 1 < len(trecho):
        trecho_pos_virgula = trecho[pos_virgula + 1:].lstrip()
        if limite < len(trecho_pos_ponto) < max:
            print("[4] Fallback: vírgula usada.")
            return trecho_pos_virgula

    # 5. Nada encontrado, retorna o trecho
    print("[5] Nenhum conectivo encontrado, retornando trecho completo.")
    return trecho.strip()

In [776]:
# def extrair_trecho_final(texto, conectivos, limite_min=60, limite_max=1000):

#     texto = texto.strip()
#     trecho_final = texto[int(len(texto) * 0.60):]
#     frases = re.split(r'(?<=[.!?])\s+', trecho_final)

#     # PRIORIDADE 1: Conectivos exatos no início de frase (formais)
#     for frase in frases:
#         frase_strip = frase.strip()
#         for c in conectivos:
#             if frase_strip.lower().startswith(c):
#                 if limite_min <= len(frase_strip) <= limite_max:
#                     print(f"[1] Conectivo exato no início: {c}")
#                     return frase_strip

#     # PRIORIDADE 2: Conectivos dentro das frases finais (mesmo no meio)
#     for frase in frases:
#         for c in conectivos:
#             if c in frase.lower():
#                 if limite_min <= len(frase.strip()) <= limite_max:
#                     print(f"[2] Conectivo encontrado no meio: {c}")
#                     return frase.strip()

#     # PRIORIDADE 3: Expressões verbais conclusivas mais amplas
#     for frase in frases:
#         for exp in expressoes:
#             if exp in frase.lower():
#                 if limite_min <= len(frase.strip()) <= limite_max:
#                     print(f"[3] Expressão conclusiva encontrada: {exp}")
#                     return frase.strip()

#     # PRIORIDADE 4: Últimas duas frases do texto 
#     if len(frases) >= 2:
#         ultimas = ' '.join(frases[-2:]).strip()
#         if limite_min <= len(ultimas) <= limite_max:
#             print("[4] Últimas duas frases usadas.")
#             return ultimas
#     elif frases:
#         ultima = frases[-1].strip()
#         if limite_min <= len(ultima) <= limite_max:
#             print("[4] Última frase usada.")
#             return ultima

#     # PRIORIDADE 5: Últimos caracteres do texto como fallback
#     fallback = texto[-limite_max:].strip()
#     print("[5] Fallback: últimos caracteres.")
#     return fallback if len(fallback) >= limite_min else texto

In [777]:
bd = pd.read_csv('conclusaoValidacao.csv')

In [778]:
redacoes = bd['Redacao']

In [779]:
def preencher_tabela(redacoes):
    dados = []

    for i, texto in enumerate(redacoes):
        texto = texto.strip()
        conclusao = extrair_trecho_final(texto,conectivos)
        QtdeCaracteres = len(conclusao)
        qtde_frases = len(sent_tokenize(conclusao, language='portuguese'))
        porcentagem = round((len(conclusao) / len(texto)) * 100, 2)

        dados.append({
            'Redação': texto,
            'QtdeCaracteres': QtdeCaracteres,
            'Intervencao': conclusao,
            'QtdeSentencasIntervencao': qtde_frases,
            'PorcentagemCaracteresIntervencao': porcentagem
        })

    return pd.DataFrame(dados)

In [780]:
dados = preencher_tabela(redacoes)

[1] Conectivo após ponto: porém
[1] Conectivo após ponto: assim
[2] Conectivo Maiúsculo encontrado: tendo em vista
[2] Conectivo minúsculo encontrado: é necessário
[3] Fallback: ponto usado.
[1] Conectivo após ponto: destarte
[2] Conectivo Maiúsculo encontrado: dessa forma
[2] Conectivo Maiúsculo encontrado: é possível concluir que
[1] Conectivo após ponto: portanto
[2] Conectivo minúsculo encontrado: logo
[3] Fallback: ponto usado.
[1] Conectivo após ponto: portanto
[1] Conectivo após ponto: assim
[3] Fallback: ponto usado.
[2] Conectivo Maiúsculo encontrado: dessa maneira
[2] Conectivo minúsculo encontrado: sendo assim
[1] Conectivo após ponto: assim
[1] Conectivo após ponto: dessarte
[1] Conectivo após ponto: portanto
[2] Conectivo minúsculo encontrado: de forma que
[1] Conectivo após ponto: assim
[2] Conectivo Maiúsculo encontrado: dessa forma
[3] Fallback: ponto usado.
[1] Conectivo após ponto: consequentemente
[1] Conectivo após ponto: assim
[1] Conectivo após ponto: portanto
[3]

In [781]:
dados.head(16)

,Redação,QtdeCaracteres,Intervencao,QtdeSentencasIntervencao,PorcentagemCaracteresIntervencao
0,A grande importância dos caminhoneiros para o ...,506,"Porém, não seria mais inteligente se o preside...",2,22.98
1,O senhor e eu já estamos cientes do projeto de...,234,"Assim, Excelentíssimo Senhor, sugiro que o sen...",2,16.36
2,É de conhecimento geral dos brasileiros o quão...,745,Tendo em vista o pedido de Impeachment e toman...,4,28.85
3,Podemos perceber que a sociedade brasileira en...,417,é necessário que o governo proporcione valoriz...,1,29.70
4,Comenta-se que o ex presidente Lula ex-preside...,311,"Mediante a sintaxe, o Ministério Público Feder...",1,24.05
5,É notório como notícias envolvendo o homossexu...,261,"Destarte, a sociedade deve compreender o senti...",2,11.53
6,É de conhecimento geral que a homossexualidade...,252,"Dessa forma, conclui-se que não existe uma cur...",1,22.54
7,Os resultados das urnas nas últimas eleições n...,470,É possível concluir que a capacidade de mudanç...,3,21.72
8,"Há alguns meses atrás , ocorreu um evento que ...",522,"Portanto, a afirmação de que esses direitos só...",3,29.26
9,"Amazonas, sem ela ele os dias do brasil Brasil...",100,logo esse incêndio ira irá ser controlado e ap...,1,11.81


In [796]:
nome_arquivo = f'conclusaoValidacaoRes.xlsx'  
dados.to_excel(nome_arquivo, index=False) 

In [783]:
from difflib import SequenceMatcher

def similaridade(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [784]:
df = pd.read_excel("diferencas_destacadas01.xlsx")

In [795]:
dados['testeLLM'] = df['testeLLM']
dados['QtdeCaracteresLLM'] = df['QtdeCaracteresLLM']
dados['QtdeSentencasIntervencaoLLM'] = df['QtdeSentencasIntervencaoLLM']
dados['PorcentagemCaracteresIntervencaoLLM'] = df['PorcentagemCaracteresIntervencaoLLM']

In [794]:
df

,Redação,QtdeCaracteres,Intervencao,QtdeSentencasIntervencao,PorcentagemCaracteresIntervencao,testeLLM,QtdeCaracteresLLM,QtdeSentencasIntervencaoLLM,PorcentagemCaracteresIntervencaoLLM
0,A grande importância dos caminhoneiros para o ...,506,"Porém, não seria mais inteligente se o preside...",2,22.98,"Porém, não seria mais inteligente se o preside...",506,2,22.98
1,O senhor e eu já estamos cientes do projeto de...,234,"Assim, Excelentíssimo Senhor, sugiro que o sen...",2,16.36,"Assim, Excelentíssimo Senhor, sugiro que o sen...",234,2,16.36
2,É de conhecimento geral dos brasileiros o quão...,745,Tendo em vista o pedido de Impeachment e toman...,4,28.85,Tendo em vista o pedido de Impeachment e toman...,745,4,28.85
3,Podemos perceber que a sociedade brasileira en...,429,"Desse modo, é necessário que o governo proporc...",1,30.56,"Desse modo, é necessário que o governo proporc...",429,1,30.56
4,Comenta-se que o ex presidente Lula ex-preside...,311,"Mediante a sintaxe, o Ministério Público Feder...",1,24.05,"Medianta a sintaxe, o Ministério Público Feder...",311,1,24.05
...,...,...,...,...,...,...,...,...,...
545,É possível notar que o Brasil é um país com um...,258,"Portanto, a fim de valorizarmos ainda mais o p...",2,20.64,erro,4,1,0.32
546,As redes sociais tem fugido do seu papel socia...,568,assim deve se manter as bases que sustenta o n...,2,29.77,erro,4,1,0.21
547,A escola foi criada para trazer mais educação ...,352,"Há projeto do governo para a educação, mas fal...",1,33.56,erro,4,1,0.38
548,"Prolongada Promulgada pela ONU em 1948, a Decl...",432,"Contudo, a realidade é justamente o oposto e o...",4,39.17,erro,4,1,0.36


In [785]:
dfT = pd.DataFrame()

In [786]:
dfT['Intervencao'] = dados['Intervencao']
dfT['testeLLM'] = df['testeLLM']

In [ ]:
dfT['similaridade'] = dfT.apply(lambda row: similaridade(str(row['Intervencao']), str(row['testeLLM'])), axis=1)

# Filtra os casos onde a similaridade é boa (por exemplo, acima de 0.80)
dfT_bons = dfT[dfT['similaridade'] >= 0.80]

print(f"Número de casos parecidos: {len(dfT_bons)} de {len(dfT)}")
acuracia = len(dfT_bons) / len(dfT) * 100
print(f"Acuracia de: {acuracia:.2f}%") 17

Número de casos parecidos: 331 de 500
Acuracia de: 66.20%


In [ ]:
dfT['similaridade'] = dfT.apply(lambda row: similaridade(str(row['Intervencao']), str(row['testeLLM'])), axis=1)

# Filtra os casos onde a similaridade é boa (por exemplo, acima de 0.80)
dfT_bons = dfT[(dfT['similaridade'] <= 0.80) & (dfT['similaridade'] >= 0.50)]

print(f"Número de casos parecidos: {len(dfT_bons)} de {len(dfT)}")
acuracia = len(dfT_bons) / len(dfT) * 100
print(f"Acuracia de: {acuracia:.2f}%")

Número de casos parecidos: 87 de 500
Acuracia de: 17.40%


In [789]:
dfT['similaridade'] = dfT.apply(lambda row: similaridade(str(row['Intervencao']), str(row['testeLLM'])), axis=1)

# Filtra os casos onde a similaridade é boa (por exemplo, acima de 0.80)
dfT_bons = dfT[dfT['similaridade'] <= 0.50]

print(f"Número de casos parecidos: {len(dfT_bons)} de {len(dfT)}")
acuracia = len(dfT_bons) / len(dfT) * 100
print(f"Acuracia de: {acuracia:.2f}%")

Número de casos parecidos: 82 de 500
Acuracia de: 16.40%


In [790]:
dfT[dfT['similaridade'] < 0.8]

,Intervencao,testeLLM,similaridade
5,"Destarte, a sociedade deve compreender o senti...","Sendo assim, é de suma importância a placidez ...",0.396861
8,"Portanto, a afirmação de que esses direitos só...","São as pessoas que mudam o seu país, e em um a...",0.613546
12,Assim diminuir ou prolongar tragédias maiores ...,Para amenizar ambas as partes prejudicadas pre...,0.423698
13,"Curiosidades pessoais: Aprender novos idiomas,...","""""""Eu acredito que os dois são muito important...",0.257778
14,"Dessa maneira, só será capaz utilizar essas te...","Dado o explícito cenário, a consideração deste...",0.016639
...,...,...,...
477,"Dessa forma, é inadmissível que tal evolução o...","Em suma, a sociedade cresceu e tende a evoluir...",0.007533
481,assim o número de beneficiados por esse direit...,"Por conseguinte, é essencial que haja formas e...",0.735000
490,"Dessa forma, é possível concluir que a atuação...","Para que isso mude, umas das soluções é desvin...",0.601134
493,"Assim, torna-se evidente que, sem estímulos, c...",Somente quando o Brasil ofertar dignidade para...,0.501767


In [791]:
from collections import Counter

# Supondo que você já tenha o DataFrame df
inicios = []

for texto in df['testeLLM'].dropna():
    palavras = texto.strip().lower().split()
    if len(palavras) >= 3:
        inicio = ' '.join(palavras[:3])  # ou use 2 se quiser: palavras[:2]
        inicios.append(inicio)

contagem = Counter(inicios)
mais_comuns = contagem.most_common(20)

# Exibe o resultado
for frase, freq in mais_comuns:
    print(f"{frase} → {freq} vezes")


portanto, medidas são → 7 vezes
tendo em vista → 5 vezes
portanto, é necessário → 5 vezes
desse modo, é → 4 vezes
em suma, a → 4 vezes
para que isso → 4 vezes
portanto medidas são → 4 vezes
levando em consideração → 3 vezes
diante do exposto, → 3 vezes
dessa forma, fica → 3 vezes
levando-se em consideração → 3 vezes
conclui-se, portanto, que → 3 vezes
é necessário que → 3 vezes
por tudo isso, → 3 vezes
sendo assim, é → 2 vezes
portanto, fica evidente → 2 vezes
por todos esses → 2 vezes
em vista dos → 2 vezes
portanto, percebe-se que → 2 vezes
não há uma → 2 vezes


In [792]:
nome_arquivo = f'TesteRes.xlsx'  
dfT.to_excel(nome_arquivo, index=False) 